In [1]:
!pip install gdown
!pip install pyarabic
!pip install farasapy

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=47a18cf31f1cd6204e450387e0af1b6e47b2d9779999ab5579950b19e68da9cd
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\38\72\83\dab5cb8321023eb1549d7a7d7f564b479ed4cf71edb70ddad6
Successfully built gdown
     ------------------------------------- 126.4/126.4 kB 63.5 kB/s eta 0:00:00


In [2]:
!gdown --id 165kzfZDsRTZAAfZKedeZiUlKzMcHNgPd

C:\ProgramData\Anaconda3\lib\site-packages\gdown\cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=165kzfZDsRTZAAfZKedeZiUlKzMcHNgPd
To: D:\Courses\Uottawa\DTI5125[EG] Data Science Applications [LEC] 20225 - Sunday\Project\summary\abstractive_summarizer-master\Arabic_stop_words.txt

  0%|          | 0.00/6.48k [00:00<?, ?B/s]
100%|##########| 6.48k/6.48k [00:00<00:00, 6.49MB/s]


In [1]:
arabic_stop_words=[]
with open ('./Arabic_stop_words.txt',encoding='utf-8') as f :
    for i in f.readlines() :
        arabic_stop_words.append(i)
        arabic_stop_words[-1]=arabic_stop_words[-1][:-1]

In [2]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.stem.isri import ISRIStemmer
import logging
import os
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)
import tensorflow as tf
from transformers import BertTokenizer

In [3]:
def data_preprocessing(data_frame):
    # clean-up: remove #tags, http links and special symbols
    data_frame['text']= data_frame['text'].apply(lambda x: re.sub(r'http\S+', '', x))
    data_frame['text'] = data_frame['text'].apply(lambda x: re.sub(r'[@|#]\S*', '', x))
    data_frame['text'] = data_frame['text'].apply(lambda x: re.sub(r'"+', '', x))

    # Remove arabic signs
    data_frame['text'] = data_frame['text'].apply(lambda x: re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '', x))

    # Remove repeated letters like "الللللللللللللللله" to "الله"
    data_frame['text'] = data_frame['text'].apply(lambda x: x[0:2] + ''.join([x[i] for i in range(2, len(x)) if x[i]!=x[i-1] or x[i]!=x[i-2]]))

    # remove stop words
    data_frame['text'] = data_frame['text'].apply(lambda x: '' if x in arabic_stop_words else x)

   
    data_frame['text']=data_frame['text'].apply(lambda x:ISRIStemmer().stem(x))

    return data_frame

In [4]:
def read_text(dir_path, fin):
    list_txt = []
    file_path = dir_path + "\\" + fin 
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file.readlines():
            list_txt.append(line)
    df = pd.DataFrame()
    df["text"] = list_txt
    df = data_preprocessing(df)
    list_txt = df.values.tolist()
    for article in list_txt:
        with open(f"{dir_path}\cleaned_{fin}", 'a', encoding='utf-8') as file:
            file.write(f"{article[0]}\n")

In [5]:
direct = 'arabic'
file_list = os.listdir(direct)
for file in file_list:
    read_text(direct,file)

In [6]:
direct = 'arabic'
file_list = os.listdir(direct)
files_list = [file for file in file_list if "cleaned" in file]
def tokenized_text(dir_path,file):
    content_list=list()
    tokenz_list = list()
    tok = BertTokenizer.from_pretrained('UBC-NLP/MARBERT')
    fin = dir_path + "\\" + file
    fin = open(fin, 'r',encoding="utf-8")
    for line in fin.readlines():
        word_pieces = tok.tokenize(line.strip())
        tokenz_list.append(word_pieces)
        new_line = " ".join(word_pieces)
        content_list.append(new_line)
    df = pd.DataFrame()
    df['content'] = content_list
    df['tokenized_content'] = tokenz_list
    return tok,df

In [7]:
files_list

['cleaned_cleaned_cleaned_test.src',
 'cleaned_cleaned_cleaned_test.tgt',
 'cleaned_cleaned_cleaned_train.src',
 'cleaned_cleaned_cleaned_train.tgt',
 'cleaned_cleaned_cleaned_val.src',
 'cleaned_cleaned_cleaned_val.tgt',
 'cleaned_cleaned_test.src',
 'cleaned_cleaned_test.tgt',
 'cleaned_cleaned_train.src',
 'cleaned_cleaned_train.tgt',
 'cleaned_cleaned_val.src',
 'cleaned_cleaned_val.tgt',
 'cleaned_test.src',
 'cleaned_test.tgt',
 'cleaned_train.src',
 'cleaned_train.tgt',
 'cleaned_val.src',
 'cleaned_val.tgt']

In [8]:
document_tokenizer, document = tokenized_text(direct,file = 'cleaned_train.src')
summary_tokenizer, summary = tokenized_text(direct,file = 'cleaned_train.tgt')

In [9]:
document["token_ids"] = document['tokenized_content'].apply(lambda row:document_tokenizer.convert_tokens_to_ids(row))
summary["token_ids"] = summary['tokenized_content'].apply(lambda row:summary_tokenizer.convert_tokens_to_ids(row))

In [10]:
inputs = document["token_ids"].values.tolist()
targets= summary["token_ids"].values.tolist()

In [11]:
document

,content,tokenized_content,token_ids
0,عد الصدق هو الحل الافضل في مثل تلك الحالات لان...,"[عد, الصدق, هو, الحل, الافضل, في, مثل, تلك, ال...","[2353, 8573, 2098, 5864, 5937, 1947, 2614, 410..."
1,خنازير الغي ##نية مخلوقات اجتماعية لذلك اذا كا...,"[خنازير, الغي, ##نية, مخلوقات, اجتماعية, لذلك,...","[48590, 6314, 2608, 41760, 24655, 5474, 2140, ..."
2,حتا ##ج الى ادخال مسار الملف للتو ##جه الى صفح...,"[حتا, ##ج, الى, ادخال, مسار, الملف, للتو, ##جه...","[26640, 1020, 2127, 26048, 16962, 14436, 20360..."
3,ب ان تجعل القاري مهتما بالشخص ##ية الرييسية وي...,"[ب, ان, تجعل, القاري, مهتما, بالشخص, ##ية, الر...","[118, 1946, 3188, 12053, 65318, 27900, 1988, 2..."
4,مثل احدى الطرق الرحيمة لمنع تزايد اسراب الاوز ...,"[مثل, احدى, الطرق, الرحيمة, لمنع, تزايد, اسراب...","[2614, 7575, 7398, 89222, 25372, 32084, 75081,..."
...,...,...,...
102200,,[],[]
102201,من الافضل ان يصاحب ##ك في عملية الولادة طبيب ا...,"[من, الافضل, ان, يصاحب, ##ك, في, عملية, الولاد...","[1939, 5937, 1946, 42194, 1012, 1947, 5921, 25..."
102202,,[],[]
102203,قد يتسبب العفن الموجود في المرح ##اض في الكثير...,"[قد, يتسبب, العفن, الموجود, في, المرح, ##اض, ف...","[2167, 26989, 45804, 15139, 1947, 7477, 2430, ..."


In [12]:
document_tokenizer.convert_ids_to_tokens([2353, 8573, 2098, 5864, 5937, 1947, 2614])

['عد', 'الصدق', 'هو', 'الحل', 'الافضل', 'في', 'مثل']

In [13]:
encoder_vocab_size = document_tokenizer.vocab_size
decoder_vocab_size = summary_tokenizer.vocab_size

# vocab_size
encoder_vocab_size, decoder_vocab_size

(100000, 100000)

#### Obtaining insights on lengths for defining maxlen

In [14]:
document_lengths = pd.Series([len(x) for x in document['tokenized_content']])
summary_lengths = pd.Series([len(x) for x in summary['tokenized_content']])

In [15]:
document_lengths.describe()

count    102205.000000
mean        232.570050
std         269.364991
min           0.000000
25%           0.000000
50%         171.000000
75%         386.000000
max        2685.000000
dtype: float64

In [16]:
summary_lengths.describe()

count    102205.000000
mean         20.262091
std          23.084321
min           0.000000
25%           0.000000
50%          17.000000
75%          33.000000
max         500.000000
dtype: float64

In [17]:
# maxlen
# taking values > and round figured to 75th percentile
# at the same time not leaving high variance
encoder_maxlen = 256
decoder_maxlen = 128

#### Padding/Truncating sequences for identical sequence lengths

In [18]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')

### Creating dataset pipeline

In [19]:
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

In [20]:
BUFFER_SIZE = 20000
BATCH_SIZE = 16

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Positional Encoding for adding notion of position among words as unlike RNN this is non-directional

In [22]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [23]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


### Masking

- Padding mask for masking "pad" sequences
- Lookahead mask for masking future words from contributing in prediction of current words in self attention

In [24]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [25]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

### Building the Model

#### Scaled Dot Product

In [26]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

#### Multi-Headed Attention

In [27]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights

### Feed Forward Network

In [28]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

#### Fundamental Unit of Transformer encoder

In [29]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


#### Fundamental Unit of Transformer decoder

In [30]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2


#### Encoder consisting of multiple EncoderLayer(s)

In [31]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x


#### Decoder consisting of multiple DecoderLayer(s)

In [32]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights


#### Finally, the Transformer

In [33]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


### Training

In [34]:
# hyper-params
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
EPOCHS = 50

#### Adam optimizer with custom learning rate scheduling

In [35]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


#### Defining losses and other metrics 

In [36]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [37]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [38]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [39]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

#### Transformer

In [40]:
transformer = Transformer(
    num_layers, 
    d_model, 
    num_heads, 
    dff,
    encoder_vocab_size, 
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size,
)

#### Masks

In [41]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask


#### Checkpoints

In [42]:
checkpoint_path = "checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


#### Training steps

In [148]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [149]:
import time
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)
    
        # 55k samples
        # we display 3 batch results -- 0th, middle and last one (approx)
        # 55k / 64 ~ 858; 858 / 2 = 429
        if batch % 429 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
      
    if (epoch + 1) % 5 == 0:
        ckpt_save_path = ckpt_manager.save()
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    
    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 11.5222
Epoch 1 Batch 429 Loss 10.3292
Epoch 1 Batch 858 Loss 9.3860
Epoch 1 Loss 8.9886
Time taken for 1 epoch: 182.31050944328308 secs

Epoch 2 Batch 0 Loss 7.6642
Epoch 2 Batch 429 Loss 7.7766
Epoch 2 Batch 858 Loss 7.7102
Epoch 2 Loss 7.6157
Time taken for 1 epoch: 174.18780636787415 secs

Epoch 3 Batch 0 Loss 6.4352
Epoch 3 Batch 429 Loss 7.0391
Epoch 3 Batch 858 Loss 6.9624
Epoch 3 Loss 6.8937
Time taken for 1 epoch: 175.47943949699402 secs

Epoch 4 Batch 0 Loss 6.3374
Epoch 4 Batch 429 Loss 6.4686
Epoch 4 Batch 858 Loss 6.4506
Epoch 4 Loss 6.4175
Time taken for 1 epoch: 174.64511561393738 secs

Epoch 5 Batch 0 Loss 6.2885
Epoch 5 Batch 429 Loss 5.9738
Epoch 5 Batch 858 Loss 5.9861
Saving checkpoint for epoch 5 at checkpoints\ckpt-1
Epoch 5 Loss 5.9814
Time taken for 1 epoch: 177.04201316833496 secs

Epoch 6 Batch 0 Loss 5.9953
Epoch 6 Batch 429 Loss 5.6155
Epoch 6 Batch 858 Loss 5.6555
Epoch 6 Loss 5.6738
Time taken for 1 epoch: 175.93753957748413 secs

Epoc

Epoch 49 Batch 858 Loss 3.6656
Epoch 49 Loss 3.7723
Time taken for 1 epoch: 167.59003138542175 secs

Epoch 50 Batch 0 Loss 3.4673
Epoch 50 Batch 429 Loss 3.5264
Epoch 50 Batch 858 Loss 3.6600
Saving checkpoint for epoch 50 at checkpoints\ckpt-10
Epoch 50 Loss 3.7555
Time taken for 1 epoch: 169.83916211128235 secs



### Inference

#### Predicting one word at a time at the decoder and appending it to the output; then taking the complete sequence as an input to the decoder and repeating until maxlen or stop keyword appears

In [43]:
def evaluate(input_document):
    
    input_document = document_tokenizer.convert_tokens_to_ids(input_document)
    input_document = [input_document]
    
    input_document = tf.keras.preprocessing.sequence.pad_sequences(input_document, maxlen=encoder_maxlen, padding='post', truncating='post')
    
    encoder_input = tf.expand_dims(input_document[0], 0)
    
    decoder_input = [summary_tokenizer.convert_tokens_to_ids("[CLS]")]
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == summary_tokenizer.convert_tokens_to_ids("[SEP]"):
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


In [44]:
document_test_tokenizer, document_test = tokenized_text(direct,file = 'cleaned_test.src')
# summary_test_tokenizer, summary = tokenized_text(direct,file = 'cleaned_test.tgt')

In [45]:
def summarize(input_document):
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = summary_tokenizer.convert_ids_to_tokens(summarized)
    summarized = summary_tokenizer.convert_tokens_to_string(summarized[1:])
    return  summarized 

In [46]:
# save tokenizers models for loading them in inference
d = {
    "document_tokenizer": document_tokenizer,
    "summary_tokenizer": summary_tokenizer
}
import pickle
with open("objects.pickle","wb") as obj:
    pickle.dump(d, obj, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
fin = open("arabic\\cleaned_train.tgt","r",encoding="utf-8")
l = list()
for line in fin.readlines():
    l.append(line)
test_summary = pd.DataFrame()
test_summary["actual_summary"] = l

In [48]:
test_summary = test_summary.iloc[0:40]
test_summary

,actual_summary
0,ل الحقيقة جربي أن تستخدمي طريقة الساندويتش كون...
1,قه بالقرب من رفاقه لفه وفر له كل الاحتياجات ال...
2,خل مسار الملف للتوجه إلى صفحة معينة على الموقع...
3,عل قارئك يميز الشخصية الرئيسية كن مستعدا لحدوث...
4,لم منع فقس البيض حدد مواقع الأعشاش استخدم زيت ...
5,رف الفرق بين حزم اللغة وحزم واجهة اللغة اذهب ...
6,رف سرعتك الحالية ابحث عن مكان مناسب ضع جدولا ض...
7,نبي الأطعمة المالحة والغنية بالصوديوم تناولي ا...
8,ل تطبيقا لهاتفك المحمول اضبط التطبيق فعل المكا...
9,تخدم الزراعة المائية للحصول على محصول أكثر بال...


In [49]:
document_test= document.iloc[0:40]
document_test

,content,tokenized_content,token_ids
0,عد الصدق هو الحل الافضل في مثل تلك الحالات لان...,"[عد, الصدق, هو, الحل, الافضل, في, مثل, تلك, ال...","[2353, 8573, 2098, 5864, 5937, 1947, 2614, 410..."
1,خنازير الغي ##نية مخلوقات اجتماعية لذلك اذا كا...,"[خنازير, الغي, ##نية, مخلوقات, اجتماعية, لذلك,...","[48590, 6314, 2608, 41760, 24655, 5474, 2140, ..."
2,حتا ##ج الى ادخال مسار الملف للتو ##جه الى صفح...,"[حتا, ##ج, الى, ادخال, مسار, الملف, للتو, ##جه...","[26640, 1020, 2127, 26048, 16962, 14436, 20360..."
3,ب ان تجعل القاري مهتما بالشخص ##ية الرييسية وي...,"[ب, ان, تجعل, القاري, مهتما, بالشخص, ##ية, الر...","[118, 1946, 3188, 12053, 65318, 27900, 1988, 2..."
4,مثل احدى الطرق الرحيمة لمنع تزايد اسراب الاوز ...,"[مثل, احدى, الطرق, الرحيمة, لمنع, تزايد, اسراب...","[2614, 7575, 7398, 89222, 25372, 32084, 75081,..."
5,ترجم حزم اللغة العادية اغلبية عناصر واجهة المس...,"[ترجم, حزم, اللغة, العادية, اغلبية, عناصر, واج...","[54463, 25100, 8368, 29642, 42803, 13797, 3412..."
6,ل ان تزيد من سرع ##تك عليك معرفة سرعة ركض ##ك ...,"[ل, ان, تزيد, من, سرع, ##تك, عليك, معرفة, سرعة...","[140, 1946, 7685, 1939, 28225, 2035, 2373, 946..."
7,مك يحتاج الصو ##ديو ##م بكميات صغيرة ولكن استه...,"[مك, يحتاج, الصو, ##ديو, ##م, بكميات, صغيرة, و...","[2452, 4574, 91402, 2769, 1017, 71505, 7441, 2..."
8,كنك استخدام تطبيقات لاظهار الهاتف وكانه يتلقى ...,"[كنك, استخدام, تطبيقات, لاظهار, الهاتف, وكانه,...","[18110, 8281, 20797, 65289, 15790, 12389, 3173..."
9,ني الزراعة المايية زراعة النباتات من دون تربة ...,"[ني, الزراعة, المايية, زراعة, النباتات, من, دو...","[4488, 28985, 36329, 20512, 43469, 1939, 2675,..."


In [50]:
import time
t=time.time()
test_df = pd.DataFrame()
test_df["tokenized_content"] = document_test["tokenized_content"]
test_df["tokenized_content"] = test_df["tokenized_content"].apply(lambda row: summarize(row) )
print(time.time()-t)

842.113249540329


In [90]:
!pip install rouge-score

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=4442f885f50f2694657b221a15831005308c574efc5ba04981a7b4eff0f2b583
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\5f\dd\89\461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [51]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1'])
m=0
for i in range(40):
    rog=scorer.score(test_df['tokenized_content'][i],test_summary["actual_summary"].iloc[0:40][i],)['rouge1'].fmeasure
    m+=rog
print(m)

0.0
